In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['yy'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [ ]:
df = pd.read_csv('post-ontology-label.csv')
df

,post,prefLabel,compulsion,obsession
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1
...,...,...,...,...
8125,"Good morning,\nI have always found that my OCD...","['surprise', 'Intrusive thoughts', 'Thought', ...",0,1
8126,The terror I felt when there were children pla...,"['OCD', 'talking', 'Intrusive thoughts', 'Thou...",0,1
8127,"Hi, I've started 2 have ocd when i was 10 whic...","['washing hands', 'object', 'fall', 'obsession...",0,1
8128,"Hi Everyone,\nA small true story that happened...","['Thought', 'drop', 'Rituals', 'OCD', 'compuls...",1,0


In [ ]:
df['post'] = df['post'].apply(process_text)
df.head()

,post,prefLabel,compulsion,obsession
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1


In [ ]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(df, test_size=test_split, stratify=df[['compulsion']].values, )
test_df,val_df = train_test_split(test_df,test_size=0.5,stratify=test_df[['compulsion']].values)
# Splitting the test set further into validation and new test sets.
# val_df = test_df.sample(frac=0.5)
# test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 6504
Number of rows in validation set: 813
Number of rows in test set: 813


In [ ]:
train_x,train_y = train_df['post'], train_df[['compulsion']]
test_x,test_y = test_df['post'], test_df[['compulsion']]
val_x,val_y = val_df['post'], val_df[['compulsion']]

#### COMPULSION MODEL

In [ ]:
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score


# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
compulsion_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the training and testing texts
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_y, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_y, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_y, dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}
# Compile the model
compulsion_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = compulsion_model.fit(
    train_encodings,
    train_labels,
    validation_data=(test_encodings, test_labels),
    epochs=3,
    batch_size=32
)

# Evaluate the model
predictions = compulsion_model.predict(test_encodings)
predicted_labels = tf.argmax(predictions.logits, axis=1)
accuracy = accuracy_score(test_labels, predicted_labels)
classification_rep = classification_report(test_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_rep)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
  5/204 [..............................] - ETA: 2:06:13 - loss: 0.6823 - accuracy: 0.5813

In [ ]:
compulsion_model.save_pretrained('hugging_face/compulsion_model')

In [ ]:
# Load the saved model
loaded_model = TFAutoModelForSequenceClassification.from_pretrained('hugging_face/compulsion_model')

#### MAKING PREDICTION ON MANUAL_LABEL

In [ ]:
manual = pd.read_csv('post-manual-label.csv')
manual['post'] = manual['post'].apply(lambda x: process_text(str(x)))
test_x = manual['post']
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = {key: np.array(value) for key, value in train_encodings.items()}

test_y = compulsion_model.predict(test_encodings)
test_y